In [1]:
import pandas as pd
import numpy as np
import os
import itertools
from itertools import product
import math

sale = pd.read_excel('1402_sales.xlsx')
Mapping = pd.read_excel('ProdMapping.xlsx')
center_region = pd.read_excel('center_factors.xlsx')

In [2]:
sale['ProductNameFA']=sale.ProductName
replace_dict = dict(zip(Mapping['DistProductName'],Mapping['ProductNameFA']))
sale['ProductNameFA'].replace(replace_dict,inplace=True)
sale['month'] = sale['SDate'].apply(lambda t: int(str(t)[4:6]))

In [3]:
# import pandas as pd
# import numpy as np
# import re
# import matplotlib.pyplot as plt
# from matplotlib.style import use


# use("ggplot")
# def remove_excess_spaces(input_string):
#     # Use re.sub() to replace multiple consecutive spaces with a single space
#     result_string = re.sub(r'\s+', ' ', input_string)
    
#     # Strip leading and trailing spaces
#     result_string = result_string.strip()
    
#     return result_string


# def clean_string(input_string):
#     # Define a regular expression pattern to match special characters (including Unicode)
#     special_chars_pattern = r'[^\w\s]+'
    
#     # Use re.sub() to replace matched special characters with spaces
#     result_string = re.sub(special_chars_pattern, ' ', input_string)
    
#     # Define a regular expression pattern to match digits (numbers)
#     numbers_pattern = r'\d+'
    
#     # Use re.sub() to remove matched digits from the result
#     result_string = re.sub(numbers_pattern, '', result_string)
    
#     return remove_excess_spaces(result_string.replace("_"," ").replace("V","").replace("ي","ی").replace("ك","ک").replace("آ","ا"))

# def remove_words(input_string, word_list):
#     for word in word_list:
#         input_string = input_string.replace(word,"")
#     return input_string

# def cleaning(input_string, word_list):
#     input_string = remove_words(input_string, word_list)
#     return clean_string(input_string)





# def make_farsi_text(x):
#     reshaped_text = arabic_reshaper.reshape(x)
#     farsi_text = get_display(reshaped_text)
#     return farsi_text

# def anote(ax1):
#     for p in ax1.patches:
#         h, w, x = p.get_height(), p.get_width(), p.get_x()
#         xy = (x + w / 2., h / 2)
#         text = f'{h:0.2f}'
#         ax1.annotate(text=text, xy=xy, ha='center', va='center',fontsize=7, rotation=90)

# def change_postfixes(name, postfix_list):
#     for postfix in postfix_list:
#         new_postfix = postfix.strip()
#         name.replace(postfix, new_postfix)
#     return name

In [4]:
# def matcher(sales):
#     postfix_list = [
#         " زاده",
#         " پور",
#         " نژاد",
#         " فر",

#     ]
#     word_list = [
#     "دکتر",
#     "داروخانه",
#     "شبانه",
#     "روزی",
#     "روزانه",
#     "بیمارستان",
#     "درمانگاه",
#     "گروه",
#     "مرکزی",
#     "مرکز",
#     "درمانی",
#     "پزشکی"]
#     pharmas = sales["Customer"].apply(clean_string)
#     pharmas = pharmas.apply(lambda w: remove_words(w, word_list))
#     pharmas = pharmas.apply(lambda name: change_postfixes(name, postfix_list))
#     sales["CustomerName_"] = pharmas
    
#     pharmas = sales["Address"]
#     pharmas = sales["Address"].fillna("").apply(clean_string)
#     pharmas = pharmas.apply(lambda w: remove_words(w, word_list))
#     sales["CustomerAddress_"] = pharmas
#     return sales

In [5]:

sales = sale.groupby(['ProductNameFA','month','DistributionCenter' , 'CustomerName' ])['NetSalesVolumePack'].sum()
sales = pd.DataFrame(sales)
sales.reset_index(inplace=True)
#center_region = center_region.drop(columns=['factor', 'factor_compl' ,'grade'])
columns_to_join_on = ['DistributionCenter']
sales_region = pd.merge(sales, center_region, on=columns_to_join_on, how='outer')
sales_region.to_excel('sales.xlsx' , index = False)

In [6]:
# distcenter spot coverage factor
# Calculate x (count of customername for a specific product in a specific month in each distribution center)
x = sales_region.groupby(['ProductNameFA', 'month', 'DistributionCenter'])['CustomerName'].count().reset_index()
x.rename(columns={'CustomerName': 'customer_count'}, inplace=True)

# Calculate y (distinct total count of customers which buy a product in a distribution center in all months)
y = sales_region.groupby(['ProductNameFA', 'DistributionCenter'])['CustomerName'].nunique().reset_index()
y.rename(columns={'CustomerName': 'total_count'}, inplace=True)

# Merge the x and y DataFrames on ProductNameFA and DistributionCenter
cover_factor_distcenters = pd.merge(x, y, on=['ProductNameFA', 'DistributionCenter'])

# Calculate the factor x/y and finalize structure
cover_factor_distcenters['center_cover_fcator'] = cover_factor_distcenters['customer_count'] / cover_factor_distcenters['total_count']

sales_regionQ = sales_region.groupby(['ProductNameFA', 'month', 'DistributionCenter', 'region'])['NetSalesVolumePack'].sum().reset_index()

cover_factor_distcenters_final = cover_factor_distcenters.merge(sales_regionQ, on=['ProductNameFA', 'month', 'DistributionCenter'], how='left')
cover_factor_distcenters_final.to_excel('cover_factor_distcenters.xlsx')

In [7]:
# region spot coverage factor
y_region = sales_region.groupby(['ProductNameFA', 'region'])['CustomerName'].nunique().reset_index()
y_region.rename(columns={'CustomerName': 'total_count'}, inplace=True)

x_region = sales_region.groupby(['ProductNameFA', 'month','region'])['CustomerName'].count().reset_index()
x_region.rename(columns={'CustomerName': 'customer_count'}, inplace=True)

cover_factor_regions = pd.merge(x_region, y_region, on=['ProductNameFA', 'region'])

# Calculate the factor x/y and finalize structure
cover_factor_regions['region_cover_fcator'] = cover_factor_regions['customer_count'] / cover_factor_regions['total_count']

sales_regionQQ = sales_region.groupby(['ProductNameFA', 'month', 'region'])['NetSalesVolumePack'].sum().reset_index()

cover_factor_regions_final = cover_factor_regions.merge(sales_regionQQ, on=['ProductNameFA', 'month', 'region'], how='left')
cover_factor_regions_final.to_excel('cover_factor_regions.xlsx')

In [8]:
# total spot coverage factor
y_total = sales_region.groupby(['ProductNameFA'])['CustomerName'].nunique().reset_index()
y_total.rename(columns={'CustomerName': 'total_count'}, inplace=True)

x_total = sales_region.groupby(['ProductNameFA', 'month'])['CustomerName'].count().reset_index()
x_total.rename(columns={'CustomerName': 'customer_count'}, inplace=True)

cover_factor_total = pd.merge(x_total, y_total, on=['ProductNameFA'])

# Calculate the factor x/y and finalize structure
cover_factor_total['region_cover_fcator'] = cover_factor_total['customer_count'] / cover_factor_total['total_count']

sales_regionQQQ = sales_region.groupby(['ProductNameFA', 'month'])['NetSalesVolumePack'].sum().reset_index()

cover_factor_total_final = cover_factor_total.merge(sales_regionQQQ, on=['ProductNameFA', 'month'], how='left')
cover_factor_total_final.to_excel('cover_factor_total.xlsx')

In [9]:
#filtering cover_factor_distcenters based on net sales volume
cover_factor_distcenters_final[cover_factor_distcenters_final['NetSalesVolumePack'] >=10] 

,ProductNameFA,month,DistributionCenter,customer_count,total_count,center_cover_fcator,region,NetSalesVolumePack
0,آگرینا,1,ارومیه,1,5,0.200000,3,25
1,آگرینا,2,ارومیه,1,5,0.200000,3,12
2,آگرینا,3,ارومیه,3,5,0.600000,3,11
3,آگرینا,4,ارومیه,1,5,0.200000,3,12
4,آگرینا,6,ارومیه,2,5,0.400000,3,20
...,...,...,...,...,...,...,...,...
5300,گنارکس,3,سمنان و ورامین,7,33,0.212121,2,348
5301,گنارکس,4,سمنان و ورامین,11,33,0.333333,2,376
5302,گنارکس,5,سمنان و ورامین,11,33,0.333333,2,116
5303,گنارکس,6,سمنان و ورامین,7,33,0.212121,2,276
